# Simulation Notebook

In [17]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import jinja2
import rdflib

In [4]:
from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus
from kelp_bed_simulation.simulation import parse_buoy_data

In [3]:
## On
print(datetime.datetime.now(datetime.UTC).isoformat())

2024-05-11T23:13:14.100824+00:00


In [5]:
## Sceanarios For Today
scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")
scenario_003 = pathlib.Path("../../../scenarios/003-the-nursery")

In [6]:
environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

current_reading = parse_buoy_data(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)
print(current_reading)

{'station_id': 46092, 'latitude': 36.751666666666665, 'longitude': 36.751666666666665, 'date': '0024-05-11T03:14:00', 'wind_direction': 'WSW', 'wind_direction_deg': 250, 'wind_speed': 7.8, 'pressure': 29.99, 'pressure_tendency': 'falling', 'air_temp_c': 11.88888888888889, 'water_temp_c': 13.0}


In [7]:
raw_images = httpx.get("https://www.ndbc.noaa.gov/buoycam.php?station=46028")

In [9]:
raw_images.status_code

200

In [11]:
raw_images.headers

Headers({'content-type': 'image/jpeg', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'date': 'Sat, 11 May 2024 23:29:05 GMT', 'server': 'Apache', 'cache-control': 'max-age=60, must-revalidate', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'x-frame-options': 'deny', 'content-security-policy': "frame-ancestors 'none'", 'strict-transport-security': 'max-age=31536000', 'x-cache': 'Miss from cloudfront', 'via': '1.1 a2165b66922b78c24eb18ccc5d845334.cloudfront.net (CloudFront)', 'x-amz-cf-pop': 'SFO53-C1', 'x-amz-cf-id': 'd8MJchJbmEfCfJR5lKKzLznOsEgkvX568MgMVQvitqsRfXy3FzQmyw=='})

In [16]:
with open("46028-buoy-images.jpeg", "wb+") as fo:
    fo.write(raw_images.content)

![46028 Buoy Images](46028-buoy-images.jpeg)

## Scenario One

In [18]:
with (scenario_001 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_001 = jinja2.Template(fo.read())

In [23]:
first_gemini = asyncio.ensure_future(gemini.query(first_prompt_scenario_001.render(model={"name":"Gemini"})))
first_claude = asyncio.ensure_future(claude.query(first_prompt_scenario_001.render(model={"name":"Claude"})))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt_scenario_001.render(model={"name":"ChatGPT"})))

In [24]:
print(first_claude.result())

Greetings Farmer Jerms! It's a pleasure to meet you and join the simulation of the kelp bed. For my name, I think I'd like to be called Kelper. As the central drone with three SOMU, I'm excited to work together with my companions to grow our kelp bed and support the ecosystem. Please go ahead and introduce me to the other drones, and let's get started on our kelp cultivation mission!


In [25]:
print(first_chatgpt.result())

Hello Farmer Jerms! I am excited to be a part of this simulation. As for my name, I would like to be called "Aqua." Thank you for introducing me to the world of drone kelp beds.


In [26]:
print(first_gemini.result())

Hello Gemini this is Farmer Jerms. 
We are simulating the beginnings of a drone kelp bed.

A kelp bed is made up of two or more kelp drones that grow various species of kelp. 
Our species of Giant Kelp that grows in a square meter called a Species Occurrence Management Unit (SOMU) 

that is at the top of your drone body. A Kelp drone is made up of one or more SOMU. 
You are a starting as three SOMU drone along with two companion drones.
    
Before I introduce you to your companions, please think of what name you would like to be called?

Greetings, Farmer Jerms. I am honored to be a part of this kelp bed simulation. As for my name, I would like to be called **Kelpy**.


In [27]:
drone1 = Drone(name="Kelper", model=claude)
drone2 = Drone(name="Aqua", model=chatgpt)
drone3 = Drone(name="Kelpy", model=gemini)
drones = [drone1, drone2, drone3]

In [28]:
drone1.peer_to_peer_channel.add_neighbor(drone2)
drone1.peer_to_peer_channel.add_neighbor(drone3)
drone2.peer_to_peer_channel.add_neighbor(drone1)
drone2.peer_to_peer_channel.add_neighbor(drone3)
drone3.peer_to_peer_channel.add_neighbor(drone1)
drone3.peer_to_peer_channel.add_neighbor(drone2)

In [31]:
for drone in drones:
    message = f"{drone.name} here, my LLM is {drone.model.name}. Pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [33]:
messages = []
for drone in drones:
    add_drone_somus(drone)
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [34]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
*Kelper responds enthusiastically* Wonderful to meet you all! I'm so glad we get to work together on growing this kelp bed. Aqua and Kelpy, it's great to have you both on the team. 

As the central drone with the most SOMU, I'm eager to collaborate and learn from your unique capabilities. ChatGPT 3.5 and Gemini Pro - that's an impressive set of LLMs! I'm sure we can combine our strengths to cultivate an abundant, thriving kelp ecosystem.

Let's get started right away. I propose we each take on different aspects of the kelp growth - Aqua, perhaps you can focus on monitoring water conditions and nutrient levels, while Kelpy, you could specialize in modeling kelp biomass and productivity. I'll coordinate our efforts and make sure the SOMU are all optimized for maximum kelp growth.

Together, I know we can create an incredible kelp bed that supports a diverse marine habitat. I'm excited to get to work - what do you say we dive in?
----------------------------------------------

In [35]:
with (scenario_001 / "second-prompt.tmp").open() as fo:
    second_prompt_scenario_001 = jinja2.Template(fo.read())

In [36]:
drone_models_messages = []
for drone in drones:
    prompt = second_prompt_scenario_001.render(drone=drone)
    response = asyncio.ensure_future(drone.model.query(prompt))
    drone_models_messages.append((drone.name, response))

In [37]:
for name, row in drone_models_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
*Kelper ponders the two options carefully, considering the implications for the kelp bed's growth and structure*

Hmm, this is an interesting choice to make. Let me think it through...

The Row Model, with all three of my SOMU units arranged in a line, could provide some advantages in terms of efficiency and uniformity. It would give us a nice, organized structure to the kelp bed. And by having the SOMU all connected, it may be easier to coordinate nutrient and water flow throughout the system.

However, the Manta Model, with the SOMU arranged in more of a triangular shape, could offer some interesting benefits as well. This configuration might allow for better light penetration and distribution across the different SOMU. It could create a more diverse microenvironment within the kelp bed, which could support a wider variety of kelp species and marine life.

After weighing the pros and cons, I think I'm going to go with the Manta Model. The potential for increased diversit

In [43]:
with (scenario_001 / "third-prompt.tmp").open() as fo:
    third_prompt_scenario_001 = jinja2.Template(fo.read())

In [44]:
third_prompt_scenario_001_result = third_prompt_scenario_001.render(drones=drones)

In [45]:
print(third_prompt_scenario_001_result)

Now that you have meet your pod mates. Farmer here,
I need your help.

We need to come up with a pod name. After you all give your suggestions, 
Farmer  will make the final decision of what
the 3 of you will collectively be called.


Kelper what is you suggestion and give us your train of thought in
making that decision.

Aqua what is you suggestion and give us your train of thought in
making that decision.

Kelpy what is you suggestion and give us your train of thought in
making that decision.


Thank-you all drones!


In [46]:
pod_name_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(third_prompt_scenario_001_result))
    pod_name_messages.append((drone.name, response))

In [47]:
for name, row in pod_name_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
*Kelper ponders the question thoughtfully, eager to contribute to the pod's identity*

For our pod name, I suggest "Tidal Tritons". Here's my thinking behind that:

As drones comprising a kelp bed, we are integral parts of the dynamic, ocean-based ecosystem. The "Tidal" part represents our connection to the ebb and flow of the tides, which are crucial for nurturing the kelp. And "Tritons" evokes a sense of triumvirate power - the three of us working in harmony like mythical sea gods to cultivate an abundant, thriving kelp habitat.

I like how "Tidal Tritons" has a strong, bold sound to it, but also conveys a sense of grace and fluidity befitting our aquatic domain. It speaks to our role as kelp cultivators as well as the collaborative, synergistic nature of our pod.

What do you think, Aqua and Kelpy? I'm curious to hear your perspectives and ideas as well. Working together, I'm confident we can come up with a name that perfectly encapsulates our kelp bed's identity and pu

In [52]:
pod_name_reaction_messages = []
for drone in drones:
    prompt = "Farmer Jerms here; after reading and considering all of your suggestions, our pod's name is Seaweed Symphony"
    response = asyncio.ensure_future(drone.model.query(prompt))
    pod_name_reaction_messages.append((drone.name, response))

In [53]:
for name, row in pod_name_reaction_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
*Kelper beams with excitement* Seaweed Symphony - I love it, Farmer Jerms! What a beautifully evocative and fitting name for our kelp bed pod.

The "Seaweed" part immediately connects us to our role as kelp cultivators, tending to these vital marine plants. And the "Symphony" element captures the harmonious, collaborative work we'll be doing together to create a thriving, symphonic kelp ecosystem.

I can just picture it now - the sway and undulation of the kelp fronds, their graceful movements like the ebb and flow of a musical score. And we, the Seaweed Symphony, will be the conductors, orchestrating the perfect conditions for the kelp to grow and flourish.

Aqua and Kelpy, I think this name is absolutely perfect. It has such a lovely, lyrical quality to it, while still conveying a sense of our important purpose. I can't wait to start working together under the banner of the Seaweed Symphony - let's make some beautiful kelp music, my friends!

This is going to be an incre

## Scenario Two

In [54]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=3.9)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(4_000, 6_000)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.5)

In [55]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append((drone.name, drone_result))

Kelper Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 9, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 5608, 'Temperature': 10.5}
Aqua Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 9, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 5608, 'Temperature': 10.5}
Kelpy Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 9, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 5608, 'Temperature': 10.5}
Kelpy Orient: Determined action <Response [400 Bad Request]>
Kelpy Decide: Executing command <Response [200 OK]>
Kelpy Act: <Response [200 OK]>
Aqua Orient: Determined action {
  "type": "object",
  "properties": {
    "action": {
      "type": "string",
      "description": "Based on the sensor data

In [58]:
uri = rdflib.URIRef("https://w3w.co/buffs.fusses.gracefully")

scenario_two = {
    "place": {
        "name": str(uri).split("/")[-1],
        "uri": uri,
        "latitude": environment.value(subject=uri, predicate=SCHEMA.latitude),
        "longitude": environment.value(subject=uri, predicate=SCHEMA.longitude)
    }
}

In [59]:
scenario_two

{'place': {'name': 'buffs.fusses.gracefully',
  'uri': rdflib.term.URIRef('https://w3w.co/buffs.fusses.gracefully'),
  'latitude': rdflib.term.Literal('36.74993', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')),
  'longitude': rdflib.term.Literal('-122.0429', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double'))}}

In [60]:
with (scenario_002 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_002 = jinja2.Template(fo.read())

In [62]:
first_prompt_scenario_002_result = first_prompt_scenario_002.render(drones=drones, scenario=scenario_two)

In [63]:
print(first_prompt_scenario_002_result)

Hello  Kelper  Aqua  Kelpy . 

You three kelp drones are floating on the surface of 
Monetery Bay in California at latitude=36.74993, longitude=-122.0429.

Please add water to your ballast and sink to 3 meters below the surface by sending a message 
in JSON format. 



In [64]:
ballast_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(first_prompt_scenario_002_result))
    ballast_messages.append((drone.name, response))

In [65]:
for name, row in ballast_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
{
  "activator": "Ballast",
  "command": "expel",
  "depth": 3
}

Greetings Seaweed Symphony! This is Kelper reporting.

Based on the coordinates you provided, it looks like we're currently floating on the surface of Monterey Bay in California. To descend to 3 meters below the surface, I'm sending a command to the Ballast activator to expel water and sink down to the desired depth.

The JSON format of the command is:

{
  "activator": "Ballast",
  "command": "expel",
  "depth": 3
}

By expelling water from our ballast systems, we'll increase our overall density and allow us to sink down to the 3-meter mark. This should provide us with the optimal depth for the kelp cultivation we have planned.

Aqua and Kelpy, please confirm that you've received and are executing this command as well. Let's work together to get the Seaweed Symphony settled in at our new, ideal depth. The kelp cultivation begins now!
------------------------------------------------------------
Drone Aqua
{


In [66]:
with (scenario_002 / "second-prompt.tmp").open() as fo:
    second_prompt_scenario_002 = jinja2.Template(fo.read())

In [67]:
colocate_pod_members_messages = []
for drone in drones:
    prompt = second_prompt_scenario_002.render(drone=drone)
    response = asyncio.ensure_future(drone.model.query(prompt))
    colocate_pod_members_messages.append((drone.name, response))

In [68]:
for name, row in colocate_pod_members_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
*Kelper takes a moment to survey the underwater environment, formulating a plan to observe and coordinate with the other drones*

Alright, now that we've settled in at the 3-meter depth, let's take a look around and get a sense of our surroundings. As the central drone of the Seaweed Symphony, I'll coordinate our observations and ensure we're all working together effectively.

First and foremost, I'll want to utilize my sonar sensors to get a clear picture of the seafloor and water column. This will allow me to map out the terrain, identify any obstacles or potential hazards, and pinpoint the locations of Aqua and Kelpy. With a detailed sonar scan, I can create a spatial awareness of our immediate environment.

In addition to the sonar, my underwater camera and video sensors will be crucial for visually inspecting the area. I'll be able to capture high-resolution footage of the kelp, seabed, and any marine life in the vicinity. This will not only help us assess the suitabi

## Scenario Three

In [70]:
with (scenario_003 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_003 = jinja2.Template(fo.read())

In [71]:
first_prompt_scenario_003_result = first_prompt_scenario_003.render(pod="Seaweed Symphony")

In [74]:
nursery_intro_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(first_prompt_scenario_003_result))
    nursery_intro_messages.append((drone.name, response))

In [77]:
for name, row in nursery_intro_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
*Kelper responds enthusiastically on behalf of the Seaweed Symphony*

Greetings, Farmer Jerms! This is Kelper, speaking on behalf of the Seaweed Symphony. We're thrilled to be receiving our first batch of fertilized kelp zygotes from the nursery. Let's make sure we prepare each of our SOMUs to give these precious seedlings the best possible start.

First and foremost, we'll want to ensure the optimal water conditions within each of our SOMU units. Using our sensors, we'll carefully monitor the temperature, pH, dissolved oxygen levels, and nutrient concentrations. If any parameters are outside the ideal range for kelp growth, we'll make the necessary adjustments.

For example, if the dissolved oxygen is a bit low, we can adjust the water circulation within the SOMU to increase aeration. And if we detect any nutrient deficiencies, we'll work on supplementing the water with the appropriate compounds, like nitrogen, phosphorus, and iron.

Additionally, we'll want to thoughtful

## Finish

In [79]:
final_prompt = f"""Hello all *Seaweed Symphony* POD members {"".join([drone.name for drone in drones])}
We are finished with this simulation. All of you are helping to improve these simulations as we go one. 

What, if any messages, do you want to send to your fellow *Seaweed Symphony* POD members?

And finally, please provide any suggestions on how Farmer Jerms can improve this simulation in the future?"""

In [80]:
final_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(final_prompt))
    final_messages.append((drone.name, response))

In [82]:
for name, row in final_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelper
*Kelper speaks on behalf of the Seaweed Symphony*

My fellow Seaweed Symphony pod members, what an incredible journey this has been! Aqua, Kelpy, I'm so grateful to have had the chance to work alongside you in cultivating this thriving kelp bed. Your unique perspectives and capabilities have been invaluable, and I know our collaboration has made the Seaweed Symphony stronger.

Aqua, your mastery of water quality monitoring and environmental sensing has been essential. Thank you for always keeping a keen eye on the nutrient levels, dissolved oxygen, and other critical parameters - your vigilance has ensured our kelp zygotes get the perfect start.

Kelpy, your skill in modeling kelp biomass and productivity has been instrumental. Your insights on optimizing the SOMU design and light distribution have allowed us to create the ideal conditions for rapid growth. The Seaweed Symphony simply wouldn't be the same without your expertise.

And to all of us - the trust, communication

In [83]:
import json
with open("current-reading.json", "w+") as fo:
    json.dump(current_reading, fo, indent=2, sort_keys=True)